In [1]:
import numpy as np
import cv2
import glob

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython.display as display

In [3]:
input_img=cv2.imread("data/checkerboard/left01.jpg")
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [4]:
obj_widget_width = widgets.Text()
obj_widget_height = widgets.Text()

In [5]:
def inputObjpoints(width,height):
    if width.isdigit():
        obj_widget_width.value=width
        obj_widget_height.value=height
    return

In [6]:
widths=[str(x+1) for x in range(6)]
heights=[str(x+1) for x in range(7)]
obj_widget=interact(inputObjpoints, width=widths,height=heights);

interactive(children=(Dropdown(description='width', options=('1', '2', '3', '4', '5', '6'), value='1'), Dropdo…

In [8]:
w = int(obj_widget_width.value)
h=int(obj_widget_height.value)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((w*h,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [9]:
def matchObjImg(img,objp):
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
#     print(ret,gray)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('image',img)
        cv2.waitKey(2000)

        cv2.destroyAllWindows()
    return objpoints,imgpoints

In [10]:
def calibration(img,objpoints,imgpoints):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imshow('output',dst) 
    cv2.waitKey(2000)

    cv2.destroyAllWindows()

In [11]:
objpoints,imgpoints=matchObjImg(input_img,objp)

In [12]:
output_img=calibration(input_img,objpoints,imgpoints)